# Initial Logistic Regression (no Cross Validation, just experiment)

## Import packages

In [1]:
import numpy as np
import pandas as pd
import mord
from sklearn import linear_model, metrics

## Read data

In [4]:
path = '../../data/cleaned'
training = pd.read_csv(path + '/training_cleaned_v2.csv')
testing = pd.read_csv(path + '/testing_cleaned_v2.csv')

In [5]:
print (training.isnull().sum())
training.columns

id                                   0
amount_tsh                           0
year_recorded                        0
month_recorded                       0
day_recorded                         0
gps_height                           0
basin                                0
basin_encoded                        0
region                               0
region_encoded                       0
population                           0
public_meeting_new                3334
permit_new                        3056
age                              20709
extraction_type                      0
extraction_type_encoded              0
extraction_type_group                0
extraction_type_group_encoded        0
extraction_type_class                0
extraction_type_class_encoded        0
management                           0
management_encoded                   0
management_group_new                 0
management_group_new_encoded         0
payment                              0
payment_encoded          

Index(['id', 'amount_tsh', 'year_recorded', 'month_recorded', 'day_recorded',
       'gps_height', 'basin', 'basin_encoded', 'region', 'region_encoded',
       'population', 'public_meeting_new', 'permit_new', 'age',
       'extraction_type', 'extraction_type_encoded', 'extraction_type_group',
       'extraction_type_group_encoded', 'extraction_type_class',
       'extraction_type_class_encoded', 'management', 'management_encoded',
       'management_group_new', 'management_group_new_encoded', 'payment',
       'payment_encoded', 'quantity_group', 'quantity_group_encoded',
       'quality_group_new', 'quality_group_new_encoded', 'source',
       'source_encoded', 'source_type', 'source_type_encoded', 'source_class',
       'source_class_encoded', 'waterpoint_type_new',
       'waterpoint_type_new_encoded', 'status_group', 'status_group_encoded'],
      dtype='object')

## Ordinal Logistic Regression

In [11]:
# just try
X = training.loc[:,('amount_tsh', 'year_recorded', 'gps_height', 'basin_encoded','region_encoded', 'population',
                    'extraction_type_class_encoded', 'management_group_new_encoded', 'payment_encoded', 'quantity_group_encoded',
                   'quality_group_new_encoded', 'source_class_encoded', 'waterpoint_type_new_encoded')]
y = training['status_group_encoded']

In [12]:
reg0 = linear_model.LogisticRegression(
    solver='lbfgs',
    multi_class='multinomial')
reg0.fit(X, y)
print (reg0.predict(X))
print('Mean Absolute Error of LogisticSE %s' %
      metrics.mean_absolute_error(reg0.predict(X), y))
print (sum(reg0.predict(X)==1))

[3 3 3 ... 3 3 3]
Mean Absolute Error of LogisticSE 0.7630471380471381
18663


In [13]:
reg1 = mord.LogisticAT(alpha=1.)
reg1.fit(X, y)
print (reg1.predict(X))
print('Mean Absolute Error of LogisticSE %s' %
      metrics.mean_absolute_error(reg1.predict(X), y))
print (sum(reg1.predict(X)==1))

[3 3 3 ... 3 2 3]
Mean Absolute Error of LogisticSE 0.7301178451178452
14642


In [14]:
reg2 = mord.LogisticIT(alpha=1.)
reg2.fit(X, y)
print (reg2.predict(X))
print('Mean Absolute Error of LogisticSE %s' %
      metrics.mean_absolute_error(reg2.predict(X), y))
print (sum(reg2.predict(X)==1))

[3 3 3 ... 3 3 3]
Mean Absolute Error of LogisticSE 0.7128114478114478
17687


In [15]:
reg3 = mord.LogisticSE(alpha=1.)
reg3.fit(X, y)
print (reg3.predict(X))
print('Mean Absolute Error of LogisticSE %s' %
      metrics.mean_absolute_error(reg3.predict(X), y))
print (sum(reg3.predict(X)==1))

[3 2 2 ... 2 2 2]
Mean Absolute Error of LogisticSE 0.866986531986532
996


In [15]:
# use reg2 on test set:

In [17]:
X_test = testing.loc[:,('amount_tsh', 'year_recorded', 'gps_height', 'basin_encoded','region_encoded', 'population',
                    'extraction_type_class_encoded', 'management_group_new_encoded', 'payment_encoded', 'quantity_group_encoded',
                   'quality_group_new_encoded', 'source_class_encoded', 'waterpoint_type_new_encoded')]
prediction_test = reg2.predict(X_test)
print (prediction_test)
print (sum(prediction_test==1))

[1 3 1 ... 3 3 3]
4528


In [18]:
# Let's get a submission!
testing['prediction_label'] = prediction_test
testing['status_group'] = np.where((testing['prediction_label'] == 1), 
                                                 'non functional', np.where((testing['prediction_label'] == 3), 'functional', 'functional needs repair'))
out = testing[['id', 'status_group']]

In [19]:
path_out = '../../data/submissions'
out.to_csv(path_out + '/out_v2.csv', index = False)

# To Do

## 1. Update data cleaning to make all categorical to numeric (Done)

## 2. Deal with NAs (mord does not seem like it), maybe fill? Then try mord more carefully and figure out what is going on for each method

## 3. stepwise try to figure out

## 4. try out other models

## 5. Maybe Try to Find/Write a CV function